# Derive the toric equations from the momentum representation

In [1]:
from sympy import *
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

In [2]:
# change variables manually depending on the problem

k, p, p1, p2, m, m1, m2, m3, nu1, nu2, nu3 = symbols("k, p, p1, p2, m, m1, m2 , m3, nu1, nu2, nu3")

# change expression to integrand of feynman integral in (post wick-transform) momentum space

f_momentum = (1/(k**2 + m3**2)**nu1) * (1/((k - p1)**2 + m2**2)**nu2) * (1/((k + p2)**2 + m1**2)**nu3) # massive triangle

Integral(f_momentum, k)

Integral(1/((k**2 + m3**2)**nu1*(m1**2 + (k + p2)**2)**nu3*(m2**2 + (k - p1)**2)**nu2), k)

In [3]:
f_schwinger = 1

t = symbols(f"t1:{len(f_momentum.args) + 1}")

for i, arg in enumerate(f_momentum.args):

    exponents = 1

    nu = arg.args[-1].args[-1] # specifically get nu
    exponents *= arg.args[0]

    f_schwinger *= t[i] ** (nu - 1) * exp(simplify(- exponents * t[i]))

i_f_schwinger = Integral(f_schwinger, k)

for t_i in t:
    i_f_schwinger = Integral(i_f_schwinger, t_i)

pprint(i_f_schwinger)

⌠ ⌠ ⌠ ⌠                                                                        ↪
⎮ ⎮ ⎮ ⎮                                 ⎛ 2     2⎞      ⎛  2           2⎞      ↪
⎮ ⎮ ⎮ ⎮   ν₁ - 1   ν₃ - 1   ν₂ - 1  -t₁⋅⎝k  + m₃ ⎠  -t₂⋅⎝m₁  + (k + p₂) ⎠  -t₃ ↪
⎮ ⎮ ⎮ ⎮ t₁      ⋅t₂      ⋅t₃      ⋅ℯ              ⋅ℯ                     ⋅ℯ    ↪
⌡ ⌡ ⌡ ⌡                                                                        ↪

↪                                        
↪  ⎛  2           2⎞                     
↪ ⋅⎝m₂  + (k - p₁) ⎠                     
↪                    dk d(t₁) d(t₂) d(t₃)
↪                                        


focus on only the terms in the exponential which will form our $\mathscr{U}$ and $\mathscr{F}$ symanziks

In [4]:
exp_args = [arg.args[0] for arg in f_schwinger.args if isinstance(arg, exp)]
symanzik_wip = - sum(exp_args)
symanzik_wip = collect(symanzik_wip.expand(), k)

symanzik_wip

k**2*(t1 + t2 + t3) + k*(-2*p1*t3 + 2*p2*t2) + m1**2*t2 + m2**2*t3 + m3**2*t1 + p1**2*t3 + p2**2*t2

In [5]:
U = symanzik_wip.coeff(k**2) # first symanzik polynomial

U

t1 + t2 + t3

complete the square

$$ak^2 + bk + c = (k - A)^2 + B + c$$

In [6]:
a = symanzik_wip.coeff(k**2)
b = symanzik_wip.coeff(k)
c = symanzik_wip.coeff(k, 0)

A, B = symbols("A, B")

sol = solve(((k - A)**2 + B) - (a*k**2 + b*k), [A, B])

A = sol[A] / U
B = sol[B] / U

B

-(p1*t3 - p2*t2)**2/(t1 + t2 + t3)

In [7]:
F_wip = B + c

F = factor(F_wip, 1/U) * U # second symanzik polynomial

F

m1**2*t1*t2 + m1**2*t2**2 + m1**2*t2*t3 + m2**2*t1*t3 + m2**2*t2*t3 + m2**2*t3**2 + m3**2*t1**2 + m3**2*t1*t2 + m3**2*t1*t3 + p1**2*t1*t3 + p1**2*t2*t3 + 2*p1*p2*t2*t3 + p2**2*t1*t2 + p2**2*t2*t3

$\mathscr{G}$ is the sum of the first and second Symanzik polynomials:

$$\mathscr{G} = \mathscr{U} + \mathscr{F}$$

In [8]:
G = U + F

G

m1**2*t1*t2 + m1**2*t2**2 + m1**2*t2*t3 + m2**2*t1*t3 + m2**2*t2*t3 + m2**2*t3**2 + m3**2*t1**2 + m3**2*t1*t2 + m3**2*t1*t3 + p1**2*t1*t3 + p1**2*t2*t3 + 2*p1*p2*t2*t3 + p2**2*t1*t2 + p2**2*t2*t3 + t1 + t2 + t3

collect like terms

In [9]:
for i, j in combinations(t, 2):
    G = collect(G, i*j)

G

m1**2*t2**2 + m2**2*t3**2 + m3**2*t1**2 + t1*t2*(m1**2 + m3**2 + p2**2) + t1*t3*(m2**2 + m3**2 + p1**2) + t1 + t2*t3*(m1**2 + m2**2 + p1**2 + 2*p1*p2 + p2**2) + t2 + t3

loop through terms of $\mathscr{G}$ and store the degree of each t term in the A matrix

In [10]:
A_np = np.zeros((len(t) + 1, len(G.args)), dtype = int) # have to use numpy array since sympy cant change 
                                                        # individual matrix entries once created

for i, term in enumerate(G.args): 
    A_np[0][i] = 1
    for j, tj in enumerate(t):
        A_np[j + 1][i] = degree(term, tj)

A = Matrix(A_np) # convert numpy array to sympy matrix

A

Matrix([
[1, 1, 1, 1, 1, 1, 1, 1, 1],
[1, 0, 0, 0, 0, 2, 1, 1, 0],
[0, 1, 0, 2, 0, 0, 1, 0, 1],
[0, 0, 1, 0, 2, 0, 0, 1, 1]])

In [11]:
kernel = A.nullspace()

kernel

[Matrix([
 [ 0],
 [ 2],
 [-2],
 [-1],
 [ 1],
 [ 0],
 [ 0],
 [ 0],
 [ 0]]),
 Matrix([
 [-2],
 [ 2],
 [ 0],
 [-1],
 [ 0],
 [ 1],
 [ 0],
 [ 0],
 [ 0]]),
 Matrix([
 [-1],
 [ 1],
 [ 0],
 [-1],
 [ 0],
 [ 0],
 [ 1],
 [ 0],
 [ 0]]),
 Matrix([
 [-1],
 [ 2],
 [-1],
 [-1],
 [ 0],
 [ 0],
 [ 0],
 [ 1],
 [ 0]]),
 Matrix([
 [ 0],
 [ 1],
 [-1],
 [-1],
 [ 0],
 [ 0],
 [ 0],
 [ 0],
 [ 1]])]

From the kernel vector(s) $\mathbf{u}$, extract $\mathbf{u_+}$ and $\mathbf{u_-}$, which correspond to the positive and negative components of $\mathbf{u}$. For every kernel vector, we then take the inner product of its decomposed vectors $\mathbf{u^+}$ and $\mathbf{u^-}$ with a partial derivative operator matrix

The lattice ideal is generated by the binomials 

$$x^{u^+_i} - x^{u^-_i},\quad i=1,...,r$$

In [12]:
x = symbols(f"x1:{len(kernel[0]) + 1}")

if len(kernel) != 0:

    I = Function("f")(*x)

    for i, vec in enumerate(kernel):

        u_plus = I
        u_minus = I

        for i in range(len(vec)):
            
            for _ in range(abs(vec[i])):
                if vec[i] >= 1:
                    u_plus = Derivative(u_plus, x[i], evaluate = False)
                elif vec[i] <= -1:
                    u_minus = Derivative(u_minus, x[i], evaluate = False)
        
        pprint(u_plus - u_minus)
        print()

else: print("kernel contains only zero vector!")


    3                                                 3                        ↪
   ∂                                                 ∂                         ↪
────────(f(x₁, x₂, x₃, x₄, x₅, x₆, x₇, x₈, x₉)) - ────────(f(x₁, x₂, x₃, x₄, x ↪
       2                                                 2                     ↪
∂x₅ ∂x₂                                           ∂x₄ ∂x₃                      ↪

↪                    
↪                    
↪ ₅, x₆, x₇, x₈, x₉))
↪                    
↪                    

      3                                                 3                      ↪
     ∂                                                 ∂                       ↪
- ────────(f(x₁, x₂, x₃, x₄, x₅, x₆, x₇, x₈, x₉)) + ────────(f(x₁, x₂, x₃, x₄, ↪
         2                                                 2                   ↪
  ∂x₄ ∂x₁                                           ∂x₆ ∂x₂                    ↪

↪                      
↪                      
↪  x₅, x₆, x₇, x₈, x₉))
↪    